# Iterative Phase Estimation (IQPE)

The **Iterative Phase Estimation** (**IPE**) algorithm allows the estimation of the phase of a unitary operator without the use of the **Quantum Fourier Transform**.

Present notebook reviews this **IPE** algorithm and the developed *IterativeQuantumPE* python class inside the **PE/iterative_quantum_pe** module of the library for implementing it.

Present notebook and module are based on the following references:

* *Dobšíček, Miroslav and Johansson, Göran and Shumeiko, Vitaly and Wendin, Göran*. Arbitrary accuracy iterative quantum phase estimation algorithm using a single ancillary qubit: A two-qubit benchmark. Physical Review A 3(76), 2007. https://arxiv.org/abs/quant-ph/0610214

* *Griffiths, Robert B. and Niu, Chi-Sheng*. Semiclassical Fourier Transform for Quantum Computation. Physical Review Letters, 17 (76), 1996. https://arxiv.org/abs/quant-ph/9511007

* *A. Y. Kitaev*. Quantum measurements and the abelian stabilizer problem. Electronic Colloquium on Computational Complexity, 3(3):1–22, 1996. https://arxiv.org/abs/quant-ph/9511026

* *Monz, Thomas and Nigg, Daniel and Martinez, Esteban A. and Brandl, Matthias F. and Schindler, Philipp and Rines, Richard and Wang, Shannon X. and Chuang, Isaac L. and Blatt, Rainer*. Realization of a scalable Shor algorithm. Science 6277 (351). 2016. https://arxiv.org/abs/1507.08852

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
import sys
sys.path.append("../")

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from libraries.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

## 1. Initial Inputs

For using the *IterativeQuantumPE* python class inside the **PE/iterative_quantum_pe** module 2 mandatory QLM objects should be provided:

* 1. Initial State: this will be the initial quantum state needed for applying the Unitary Operator whose phase we want to estimate.
* 2. Unitary operator: the operator whose phase we want to estimate.

For explain how the *IterativeQuantumPE* class works we are going to use the **IQPE** example from Qiskit textbook:

https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb

We are going to reproduce the section **IPE example with a 1-qubit gate for U** from Qiskit example

In [ ]:
#Number Of Qbits
n_qbits = 1

### 1. Initial State

Initial State can be:
1. QLM QRoutine
2. QLM gate (or abstract gate)

In the qiskit example the initial state will be $|1\rangle$. 

Following cell creates this initial state:

In [ ]:
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state

### 2. Unitary operator.

Unitary operator can be:

1. QLM QRoutine
2. QLM gate (or abstract gate)

In the qiskit example the unitary operator is the $\mathcal{S}$ gate. So the application of the unitary operator over the initial state will be:

$$\mathcal{S}|1\rangle = e^{i\frac{\pi}{2}}|1\rangle$$

In [ ]:
unitary_operator = qlm.PH(np.pi/2.0)  

In [ ]:
%qatdisplay unitary_operator

## 2. Class IQPE: algorithm step by step 

The problem of phase estimation can be stated as follows. Given an initial state $\left|\Psi \right\rangle$ and a phase operator $\mathcal{P}$ such that:

$$\mathcal{P}\left|\Psi \right\rangle = e^{2\pi i\lambda}\left|\Psi \right\rangle,$$

our goal is estimating $\lambda$.

So far we have the initial state $\left|\Psi \right\rangle = |1\rangle$ and the unitary operator whose phase we want to estimate $\mathcal{P} = \mathcal{S}$. In this section we are going to describe the class step by step and explain the basics of the **IPE** algorithm

### 2.1 Calling the **IQPE** class

The *IterativeQuantumPE* is inside **PE/iterative_quantum_pe** module. 

In order to instantiate the class we need to provide a pyhton dictionary. Mandatory keys are:

* initial_state : QLM routine or gate with an initial state $|\Psi\rangle$ was loaded. 
* unitary_operator :  QLM gate or routine with an Unitary operator ready for be applied to initial state $|\Psi\rangle$.

Other important keys are:

* cbits_number : int with the number of classical bits needed for for phase estimation
* qpu : QLM solver. If not provided class try to creates a PyLinalg solver. It is recomended give this key to the class.
* shots : int number of shots for quantum job.

In [ ]:
#Load Class
from libraries.PE.iterative_quantum_pe import IterativeQuantumPE

In [ ]:
n_cbits = 2
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
}
IQPE = IterativeQuantumPE(**iqpe_dict)

When the class is instantiated the properties *initial_state* and *q_gate* are overwritten with the given keys **initial_state** and **unitary_operator** respectively

In [ ]:
c = IQPE.initial_state
%qatdisplay c

In [ ]:
a = IQPE.q_gate
%qatdisplay a --depth 2

### 2.2 IPE Algorithm step by step

Now we are going to review step by step the **IPE** algorithm using different programed methods of the **IterativeQuantumPE** class

### 2.2.1. Initialize the quantum program.

First thing is calling the method **init_iqpe**. Following actions are done by this method:
1. Creation of QLM program from *initial_state* QLM routine (or AbstractGate). The QLM program is stored in *q_prog* property.
2. Allocation of an auxiliar qbit mandatory for the **IPE** algorithm. It is stored in the *q_aux* property.
3. Allocation of the auxiliar classical bits where the estimated phase will be stored. Property: *c_bits*.

In [ ]:
#Initialize the quantum program
IQPE.init_iqpe()

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
#Additionally a auxiliar qbit bits was allocated
circuit = IQPE.q_prog.to_circ(submatrices_only=True)

%qatdisplay circuit --depth 0 --svg

### 2.2.2. IPE Algorithm

We are going to decomposed the **IPE** algorithm in 2 parts. A first part where the main variable $l$ will be 0 ($l=0$) and a second recursive part where the variable $l$ will be greater than 0 ($l\gt 0$).

#### First Part ($l=0$)

The first step of the IPE algorithn ($l=0$) has the following parts:

1. Reset the auxiliar qbit
2. Apply a Haddamard gate to the auxiliar qbit
3. Apply the phase operator $\mathcal{P}$ controlled by auxiliar the qbit $2^{m-1}$ ($\mathcal{P} ^{2^{m-1}}$) times. Here $m$ is the number of classical qbits allocated for estimating $\theta$ 
4. Apply a Haddamard gate to the auxiliar qbit
5. Measuring the auxiliar qbit and store the result into classical bit array in position $c_l$ ($l=0$)

This can be done by calling the *step_iqpe* method with following arguments:

* Quantum Program with initial_state
* Quantum Routine or AbstractGate with phase operator $\mathcal{P}$
* Auxiliar Qbit
* Auxiliar classical bits
* l=0

This methods return the quantum program with the operations explained in this part.

In [ ]:
q_program = IQPE.q_prog
q_program = IQPE.step_iqpe(q_program, IQPE.q_gate, IQPE.q_aux, IQPE.c_bits, 0)

Following cell show the first  part of the circuit

In [ ]:
c = q_program.to_circ()
%qatdisplay c

#### Second or iterative Part ($l \gt 0$)

This part of the algorithm is recursive. Following steps will be repeated for each value of $l=1,2,..m-1$:

1. Reset the auxiliar qbit
2. Apply a Haddamard gate to the auxiliar qbit
3. Apply the phase operator $\mathcal{P}$ controlled by the auxiliar qbit $2^{m-1-l}$ ($\mathcal{Q}^{2^{m-1-l}}$) times, being $m$ is the number of classical qbits allocated for estimating $\theta$.
4. Apply on the auxiliar qbit a set of controlled rotations by $c_j$ classical bit of angle: $\frac{\pi}{2}\frac{1}{2^{l-j-1}}$ with $j=0,1,..l-1$. 
4. Apply a Haddamard gate to the auxiliar qbit
5. Measuring the auxiliar qbit and store the result into classical bit array in position $c_l$

So for a $l$ step de controlled by classical bits rotation will depend on the measurements don on the before $l$ steps.

In the following cell we explain how to create the algorithm for the $l=1$ part:

In [ ]:
#here we do the step l=1
l=1
q_program = IQPE.step_iqpe(q_program, IQPE.q_gate, IQPE.q_aux, IQPE.c_bits, l)

Now we can plot the circuit we have unitl the moment

In [ ]:
#Circuit for l=0 and l=1
c = q_program.to_circ()
%qatdisplay c

####  Complete algorithm

For a complete **IPE** algorithm following steps shold be done:

1. Create the First part of the algorithm $l=0$.
2. Iterate the second part of the algorihtm from  $l=1$ to $l=m-1$.

The measured classical bits is used for estimating the phase autovalues of the unitary operator

Following cell create the complete program for **IPE** algorithm

In [ ]:
#Initialize the quantum program
IQPE.init_iqpe()
q_program = IQPE.q_prog
for l in range(len(IQPE.c_bits)):
    q_program = IQPE.step_iqpe(q_program, IQPE.q_gate, IQPE.q_aux, IQPE.c_bits, l)

So for the desired number of classical bits $m$ the complete circuit for *IPE* algorithm will be:

In [ ]:
c = q_program.to_circ()
%qatdisplay c

### 2.2.3. IPE Algorithm execution

Once the QLM program is constructed the alogrithm should be executed. For this the *run* method from the class allow to execute it. Following arguments should  be provided:

* q_prog : with the complete *IPE* algorithm
* q_aux : the auxiliar qbit 
* shots 
* linalg_qpu: QLM solver

This method creates the circuit, the asociated job and execute it. The raw results of the simulation are returned


In [ ]:
raw_results = IQPE.run(q_program, IQPE.q_aux, 100, linalg_qpu=linalg_qpu)

In [ ]:
raw_results

### 2.2.4. IPE: getting classical bits measurements

As explained the phase will be estimated by getting the measurements of the classical bits. In the class this is done by the **meas_classical_bits** method. The input of this method is the *raw_results* from the *run* method. And the output will be a pandas DataFrame with the measurement of the classical bits with the following columns:

* **BitString**: is the result of the clasical bits measurement in each step of the algorithm
* **BitInt**: integer representation of the **BitString**
* **Phi**: is the estimated obtained phase and it is computed as: $\frac{BitInt}{2^{m}}$ where $m$ is the number of classical bits used for phase estimation

In [ ]:
classical_bits = IQPE.meas_classical_bits(raw_results)

In [ ]:
classical_bits

From *classical_bits* pdf the important column is **Phi**. Following the Qiskit example this column is $\varphi$ and the searched phase is: $\phi=2\pi\varphi$. In the qiskit example $\varphi=0.25$. 

The *classical_bits* DataFrame gives a result for each execution of the circuit (variable *shots*). We can obtain a frequency count of this dataframe using *sumarize* method. Following inputs should be provided:

* InputPDF: DataFrame pandas DataFrame
* columns : list with the columns user want to get the frequency

So we can obtain a summary of the results for our *Phi* column. The most frequent value should be the Qiskit result: $\varphi=0.25$ (this can be seen in folowing cell)

In [ ]:
IQPE.sumarize(classical_bits, ['Phi'])

### 2.2.5. IPE: post proccessing

Typically the phase in radians is provide in this kind of phase estimation problems. For getting this result we use the *post_proccess*. The *classical_bits* shold be provided and the output will be another DataFrame wiht following columns:

* **BitString**: is the result of the clasical bits measurement in each step of the algorithm
* **BitInt**: integer representation of the **BitString**
* **Phi**: is the estimated obtained phase and it is computed as: $\frac{BitInt}{2^{c_b}}$ where $c_b$ is the number of classical bits 
* **2*theta**: this is the phase of the unitary operator in radians:$2\theta = 2\pi\varphi$. In this case we calculate the phase of the unitary operator as the double of an angle $\theta$.
* **theta**: this is the halve of the phase of the unitary operator: $\theta = \pi\varphi$
* **theta_90**: is the $\theta$ between $(0, \frac{\pi}{2}$)

In this module our convention when $|\Psi\rangle $ is an eigenvalue of an unitary operator $\mathcal{Q}$ is:

$$\mathcal{Q}|\Psi\rangle = e^{2i\theta}|\Psi\rangle$$

In [ ]:
final_results = IQPE.post_proccess(classical_bits)

In [ ]:
final_results

The *sumarize* method can be used with the **final_results** dataframe too for getting frequency counts!!

In [ ]:
#frequencies for the column theta_90
IQPE.sumarize(final_results, ['theta_90'])

In [ ]:
#Freqeuncy for all columns
IQPE.sumarize(final_results, list(final_results.columns))

## 3. Class IQPE: complete execution

All the *IPE* steps explained in section 2 can be done with the method **iqpe**. When using this *method* following properties are populated:

* *classical_bits*: the DataFrame with the result of the *meas_classical_bits* method.
* *final_results*: the DataFrame with the result of the *post_proccess* method.
* *sumary*: the DataFrame with the result of the *sumarize* method.

In [ ]:
n_cbits = 2
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_ = IterativeQuantumPE(**iqpe_dict)

In [ ]:
iqpe_.iqpe()

In [ ]:
iqpe_.final_results

In [ ]:
iqpe_.sumary

### another example

We are going to reproduce now the qiskit example under the section *IPE example with a 2-qubit gate* (in https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb). 

In this case they use 2 qbits and want to estimate the phase for a unitary operator $\mathcal{cT}$ operator. this operator adds a $\frac{\pi}{4}$ phase to state $|11\rangle$ and leave unchanged other states. 


In [ ]:
#create initial state
n_qbits = 2
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state

In [ ]:
#Create cT operator
unitary_operator = qlm.QRoutine()
uq_qbits = unitary_operator.new_wires(n_qbits)
unitary_operator.apply(qlm.PH(np.pi/4.0).ctrl(), 0, 1)

In [ ]:
%qatdisplay unitary_operator

In [ ]:
#now IPE!!
n_cbits = 3
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_ = IterativeQuantumPE(**iqpe_dict)

In [ ]:
iqpe_.iqpe()

In [ ]:
iqpe_.sumarize(iqpe_.classical_bits, ['Phi'])

The result for $\varphi=0.125$ that is the value obtained in the Qiskit example.

## 4. Application to Amplitude Estimation

The problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $\sqrt{a}$.

For showing how *Iterative Quantum Phase Estimation* works for doing *Amplitude Estimation*, we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sqrt{a}|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$


In [ ]:
from libraries.DL.data_loading import load_probability
from libraries.AA.amplitude_amplification import grover
from libraries.PE.iterative_quantum_pe import IterativeQuantumPE

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

Next we will show how the Phase Estimation problem relates to the Amplitude Estimation problem:
$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow |\Psi\rangle\\
    & \mathcal{P} \longrightarrow \mathcal{G}
    \end{array}
$$
The first equation means that, in the phase estimation context, the initial state is $|\Psi\rangle$ and the phase operator is $\mathcal{G}$, the Grover operator corresponding to our amplitude estimaiton problem. In the next cell we define the grover operator for our problem.  

In [ ]:
target = [0, 0, 1]
index = range(oracle.arity)
grover_gate = grover(oracle, target, index)
%qatdisplay grover_gate --depth 0 --svg

Here we have used that our target state $|1\rangle$ in binary representation is $001$. See notebook *02_AmplitudeAmplification_Operators* for more information about building Grover operators.

Now that we have translated our amplitude amplification probelm to an phase estimation problem we proceed to use our class normally. We provide the *oracle* as the **initial_state** and the correspondient Grover-like operator as the **unitary_operator**. Additionally the number of classical bits (**cbits_number**) for estimating the phase should be provided.

In [ ]:
n_cbits = 8
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': oracle,
    'unitary_operator': grover_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_ = IterativeQuantumPE(**iqpe_dict)
iqpe_.iqpe()

In [ ]:
iqpe_.final_results

We are going to use the *sumary* property for getting the phase $\theta$ that is the solution of the phase estimation problem.

In [ ]:
iqpe_.sumary

Last, we use the mapping $a = \cos(\theta)^2$ to obtain the result of our amplitude estimation problem.

In [ ]:
iqpe_.sumary['P_Psi_1']=np.cos(iqpe_.sumary['theta_90'])**2

In [ ]:
iqpe_.sumary

In [ ]:
print("Classical result: ",probability[1])
print("Quantum result: ",iqpe_.sumary['P_Psi_1'].iloc[0])

In [ ]:
print('Test OK: ', probability[1]- iqpe_.sumary['P_Psi_1'].iloc[0] < 0.005)